In [1]:
import pandas as pd
import numpy as np

import json
import requests
import urllib.parse

import spotipy.util as util

import matplotlib.pyplot as plt
import seaborn as sns

import datetime

import PIL.Image
from pylab import *
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import markovify

In [2]:
client_id = '################################'         #<----------------- YOUR ID HERE
    
client_secret = '5f71633e64584a8889e3a0d40b42cb06'       #<----------------- YOUR SECRET HERE

username = '31mj43jei7fqz5bbukh5ee5bkhf4'                                #<----------------- YOUR USERNAME HERE

scope = 'playlist-modify-public playlist-modify'

redirect_uri = 'http://google.com/'

token = util.prompt_for_user_token(username=username, 
                                   scope=scope, 
                                   client_id=client_id,   
                                   client_secret=client_secret,     
                                   redirect_uri=redirect_uri)

# Search for All Podcasts Related to Data Science

In [19]:
# Search Spotify’s catalog for content: 

# The search API now returns podcast shows and episodes. 
# Set the type parameter to show or episode to query these new types of content.



# GET https://api.spotify.com/v1/search 

token

'BQBpmQeD-jaxBZB1Ew0QA0Od557kqeYxBItZmqoe1UKVpjIWcDAI6ECq7FR-ZwT7eV4omPNbyvIncQQqt_nA9lVAZ0ekEabhEk5uj43gIpQ39mwrZSiXRWz9prfg43KmkcE7e9ZIIT-XOnYJafdsplOUKuTPkB5UaBAezN1UDJEZystmU0nswoi8x9nkki9ioup8yw'

### Enter a term to search below

In [36]:
# enter term to search here

search = 'relationship'

In [37]:
# search endpoint

endpoint_url = "https://api.spotify.com/v1/search?"


In [38]:
# PERFORM THE QUERY

id_list = []
name_list = []                                         # create empty lists to hold data
desc_list = []

type = 'show'    
market  = 'US'
limit = 50                                             # assign variables for search query
offset = 0                                             # start offset at 0


more_runs = 1                                          # instantiate conditional variables
counter = 0  

                                                            # max offset is 1000 including limit
while((offset <= 950) & (counter <= more_runs)):           # while loop to run with conditional variables



    query = f'{endpoint_url}'
    query += f'&q={search}'
    query += f'&type={type}'
    query += f'&offset={offset}'                       # format search query with assigned variables
    query += f'&market={market}'
    query += f'&limit={limit}'


    response = requests.get(query,                                           # get request
                   headers={"Content-Type":"application/json", 
                            "Authorization":f"Bearer {token}"})  
    json_response = response.json()                                           # as a json file


    for i in range(len(json_response['shows']['items'])):                      # loop through json

        id_list.append(json_response['shows']['items'][i]['id'])               # pull out info from json
        name_list.append(json_response['shows']['items'][i]['name'])           # into empty lists
        desc_list.append(json_response['shows']['items'][i]['description'])
        
        
    more_runs = (json_response['shows']['total'] // 50 )            # how many more runs of 50 are needed?       
        
    counter += 1                                                    # increase conditional counter by 1
    
    offset = offset + 50                                            # increase offset by 50

                                                                    # rinse and repeat!

In [39]:
# set up a dataframe from the lists

podcasts = pd.DataFrame()

podcasts['id'] = id_list
podcasts['name'] = name_list
podcasts['description'] = desc_list

In [40]:
podcasts

,id,name,description
0,3f69XylLpFzXs49BUXSN86,Relationship Doctor,Dr. Rachel Vanderbilt is a relationship scient...
1,31O8IX7WlZBup23cFlxFYB,Relationship Advice,I Do Podcast interviews today’s most successfu...
2,7ept4XBP60x9BZyVXWXig2,Relationship Theory,Join power couple Tom and Lisa Bilyeu as they ...
3,7qcKqLfy6UNddBGFD2lYUX,Let’s Get Vulnerable: Relationship and Dating ...,Are you ready to take the mystery out of havin...
4,1YMiWYFxVYShEKzMYSztpd,Relationship Anxiety,Welcome to the Relationship Anxiety podcast! T...
...,...,...,...
994,7Kh1FjKo958YiuTxCGwERM,The Love Brain,A podcast dedicated to sharing healthy relatio...
995,1AWYHKYBkGiqPwuMGyCezu,Great Moments in Weed History,Great Moments in Weed History delves deep into...
996,1OcRVTJGLOdECYSS5CyrDz,Relationships,"A podcast on relationships by Eddy, Thien-Anh,..."
997,4sw7e6GBLoasFSfT8oFWST,"""Relationships Guided By God""","In this podcast, I will discuss relationships ..."


### Get a full list of show ID's to use later

In [41]:
# get a list of all show ids for later

show_list = list(podcasts['id'])

# List All Episodes for Multiple Shows

In [42]:
# List episodes for a show: 


# GET https://api.spotify.com/v1/shows/{id}/episodes

In [43]:
# PERFORM THE QUERY

show_id_list = []                                       # create empty lists to hold data
id_list = []
dur_list = []
date_list = []
name_list = []
desc_list = []

for show_id in show_list:                               # iterate through every show in show_list
    
    more_runs = 1                                       # instantiate conditional variables
    counter = 0    
    
    id = show_id                                        # assign variables for search query
    type = 'episodes'
    market  = 'US'                                      # start the offset at 0
    limit = 50
    offset = 0                                

                                                             # max offset is 2000 including limit
    while((offset <= 1950) & (counter <= more_runs)):        # while loop to run with conditional variables
    
        endpoint_url = f"https://api.spotify.com/v1/shows/{id}/episodes?"

        query = f'{endpoint_url}'
        query += f'&q={search}'                              # format the query with assigned variables
        query += f'&type={type}'
        query += f'&offset={offset}'
        query += f'&market={market}'
        query += f'&limit={limit}'

        response = requests.get(query,                                      # send a get request
                       headers={"Content-Type":"application/json", 
                                "Authorization":f"Bearer {token}"})
        json_response = response.json()                                     # get response as a json file
        

        offset = offset + 50                                     # increase the offset by 50
        counter += 1                                             # increase the counter by 1
        

        if next(iter(json_response)) != 'error':                 # if there wasn't any errors append data
        
            for i in range(len(json_response['items'])):                        # loop through json

                show_id_list.append(show_id)
                id_list.append(json_response['items'][i]['id'])                 # pull out info from json
                dur_list.append(json_response['items'][i]['duration_ms'])       # append into empty lists
                date_list.append(json_response['items'][i]['release_date'])    
                name_list.append(json_response['items'][i]['name'])
                desc_list.append(json_response['items'][i]['description'])
                
            more_runs = (json_response['total'] // 50 )              # how many more runs of 50 are needed?  
        
        else:                                                   # if there was an error exit the while loop                    
            offset = 1000000
            print(json_response, ' for show id: ', show_id)     # print any show_ids that gave errors
    

ConnectionError: HTTPSConnectionPool(host='api.spotify.com', port=443): Max retries exceeded with url: /v1/shows/5GwTtfpqqk4tl3KJI1F1oz/episodes?&q=relationship&type=episodes&offset=100&market=US&limit=50 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000019CC81FCAC0>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [ ]:
# setup a dataframe from the lists

all_episodes = pd.DataFrame()

all_episodes['show_id'] = show_id_list
all_episodes['episode_id'] = id_list
all_episodes['length(ms)'] = dur_list
all_episodes['date'] = date_list
all_episodes['episode_name'] = name_list
all_episodes['description'] = desc_list 

### Insert corresponding Show Name for every Episode

In [ ]:
# create a dictionary of show ids and show names

fmap = podcasts.groupby('id')['name'].apply(list).to_dict()

In [ ]:
# map dictionary to show id in dataframe

all_episodes['show_id'] = all_episodes['show_id'].map(fmap)

In [ ]:
# rename the show id column to show name

all_episodes.rename(columns = {'show_id':'show_name'}, inplace = True)

In [ ]:
# remove the brackets [] from each show name

all_episodes['show_name'] = all_episodes['show_name'].str[0]

In [ ]:
all_episodes

In [ ]:
# save dataframe to csv file

all_episodes.to_csv('relationship_podcasts.csv')